# Multi-Layer Perceptron for Text Classification

In this notebook, we try to classify brown corpus regarding its categories.

In [ ]:
%pylab inline

In [ ]:
import nltk
from nltk.corpus import brown
corpus = [(brown.words(fileid), brown.categories(fileid)) for fileid in brown.fileids()]

In [ ]:
import random
random.seed(0)
random.shuffle(corpus)

In [ ]:
docs = [' '.join(words) for words, cats in corpus]
cats = [' '.join(cats) for words, cats in corpus]

Each document is represented by a word frequency vector.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_df=0.8, min_df=3)
vecs = vectorizer.fit_transform(docs)

To process in PyTorch, we need to convert the string representation of category into a number representation.
This code assigns different IDs to each category.

In [ ]:
cat_to_id = dict()
id_to_cat = dict()
for i, cat in enumerate(brown.categories()):
    cat_to_id[cat] = i
    id_to_cat[i] = cat

In [ ]:
cat_ids = [cat_to_id[cat] for cat in cats]

In [ ]:
print(cat_ids)

The input dimension of MLP is the vocabulary size, and the output dimension is the number of classes.

In [ ]:
features = vectorizer.get_feature_names()
input_dimension = len(features)
output_dimension = len(brown.categories())

In [ ]:
print(input_dimension)
print(output_dimension)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In PyTorch, all the variables in a computational graph should be represented as `Tensor` object.
We convert the arrays of word frequency vectors and categories into the `Tensor` objects by the following code.

In [ ]:
xs = torch.FloatTensor(vecs.toarray())
ys = torch.LongTensor(cat_ids)

In [ ]:
xs_train = xs[:450]
ys_train = ys[:450]
xs_test = xs[450:]
ys_test = ys[450:]

### Model Definition

Three-layered perceptron applies two linear transformations sequentially to feature vector $x$ to obtain a prediction of $y$.
PyTorch provides `Sequential` type that allows us to make a complex model by combining some transformation functions sequentially.
As we have seen in the previous chapter, each `nn.Linear` object contains parameters $w$ and $b$.
Therefore, the model we defined contains four kinds of parameters, $w^{(1)}$ and $b^{(1)}$ for the first linear transformation and $w^{(2)}$ and $b^{(2)}$ for the second linear transformation.

In [ ]:
hidden_dimension = 10

model = torch.nn.Sequential(
    torch.nn.Linear(input_dimension, hidden_dimension),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_dimension, output_dimension)
)

In [ ]:
for param in model.parameters():
    print(param.data)

There is another way to define the model. The following code defines the same model to above. I recommend this way because you can develop your model more flexible. In the `__init__` method, we define layers **that contain model parameters to be trained**. In the `forward` method, we construct a computational graph just by writing the flow of calculation for the output.

In [ ]:
class TwoLayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TwoLayerPerceptron, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self,x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x

In [ ]:
model = TwoLayerPerceptron(input_dimension, hidden_dimension, output_dimension)

In [ ]:
for param in model.parameters():
    print(param.data)

Then we apply the cross entropy loss because we work on multi-class classification. (see https://pytorch.org/docs/master/nn.html#torch.nn.CrossEntropyLoss)

We can make the training code as usual even though we are using a compound model.

In [ ]:
cross_entropy = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)
for epoch in range(300):
    zs = model(xs_train)
    loss = cross_entropy(zs, ys_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
for param in model.parameters():
    print(param.data)

The probability of each class can be obtained by applying the softmax function to the output of the model.
We then take the class that has the maximum probability by `max()` method.

In [ ]:
ys_prob, ys_pred = F.softmax(model(xs_test), 1).max(1)

In [ ]:
ys_prob, ys_pred

Just compare with the true categories and calculate the classification accuracy.

In [ ]:
ys_test

In [ ]:
ys_test == ys_pred

In [ ]:
print((ys_test == ys_pred).sum().double() / len(ys_test))

It seems not to be so much high accuracy... But just remind that this task is the classification task into 15 classes, so we should compare this with a random guess or majority vote.

In [ ]:
ys_test_numpy = ys_test.numpy()

In [ ]:
random_guess = np.random.randint(15, size=len(ys_test))
print((ys_test_numpy == random_guess).sum() / len(ys_test))

In [ ]:
bincount = np.bincount(ys_train.numpy())
bincount

In [ ]:
majority_in_train = np.argmax(bincount)
majority_in_train

In [ ]:
majority_vote = [majority_in_train for _ in ys_test]
print((ys_test_numpy == majority_vote).sum() / len(ys_test))